In [72]:
!pip install -q langchain_groq langchain-core requests dotenv

In [73]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
groq_token = os.getenv("test_groq")
llm = ChatGroq(
    groq_api_key=groq_token,
    model_name="meta-llama/llama-4-scout-17b-16e-instruct"
)

# Use llm directly - no need for ChatHuggingFace wrapper
response = llm.invoke("hi")
print(response)

content="Hello! How can I assist you today? Do you have a specific question, topic you'd like to discuss, or task you need help with? I'm here to help!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 11, 'total_tokens': 46, 'completion_time': 0.092336168, 'prompt_time': 0.002755069, 'queue_time': 0.224692538, 'total_time': 0.095091237}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--d3eb95d1-f53f-44bb-af0b-ca32947b051f-0' usage_metadata={'input_tokens': 11, 'output_tokens': 35, 'total_tokens': 46}


In [74]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

# **Tool create**

In [75]:
#tool create

@tool
def multiply(a:int,b:int)->int:
  """Given 2 numbers a and b this tool return their product"""
  return a*b

print(multiply.invoke({'a':4,'b':5}))

20


## **Tool binding**

In [76]:
#tool binding
llm_with_tools=llm.bind_tools([multiply])

In [77]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7bae1d5fb890>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7bae1d7c2d50>, model_name='meta-llama/llama-4-scout-17b-16e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))

# **Tool Calling**

In [78]:
llm_with_tools.invoke("hi what's up?")

AIMessage(content='Not much! Just here and ready to help. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 684, 'total_tokens': 702, 'completion_time': 0.048459704, 'prompt_time': 0.076111601, 'queue_time': 0.21948551600000002, 'total_time': 0.124571305}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_79da0e0073', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7f2bc30a-e3c6-4b05-8b50-ad7d49684168-0', usage_metadata={'input_tokens': 684, 'output_tokens': 18, 'total_tokens': 702})

In [79]:
response = llm_with_tools.invoke(
    "Multiply the integers 78 and 96. Do NOT format them as strings."
)
print(response.tool_calls)

[{'name': 'multiply', 'args': {'a': 78, 'b': 96}, 'id': 'nqfgdbe56', 'type': 'tool_call'}]


# **Tool Execution**

In [80]:
response.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 78, 'b': 96},
 'id': 'nqfgdbe56',
 'type': 'tool_call'}

In [81]:
response.tool_calls[0]['args']

{'a': 78, 'b': 96}

In [82]:
multiply.invoke(response.tool_calls[0]['args'])

7488

In [83]:
multiply.invoke(response.tool_calls[0])

ToolMessage(content='7488', name='multiply', tool_call_id='nqfgdbe56')

In [84]:
query=HumanMessage("can you multiply 3 with 10 .Do Not format them as string")

In [85]:
messages=[query]
messages

[HumanMessage(content='can you multiply 3 with 10 .Do Not format them as string', additional_kwargs={}, response_metadata={})]

In [86]:
result=llm_with_tools.invoke(messages)

In [87]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '5jb87x0a0', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 693, 'total_tokens': 721, 'completion_time': 0.069178622, 'prompt_time': 0.049800984, 'queue_time': 0.56766585, 'total_time': 0.118979606}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3760a355-9c6c-4a01-80fa-3376ba4055ea-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': '5jb87x0a0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 693, 'output_tokens': 28, 'total_tokens': 721})

In [88]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 10},
  'id': '5jb87x0a0',
  'type': 'tool_call'}]

In [89]:
messages.append(result)
len(messages)
messages

[HumanMessage(content='can you multiply 3 with 10 .Do Not format them as string', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '5jb87x0a0', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 693, 'total_tokens': 721, 'completion_time': 0.069178622, 'prompt_time': 0.049800984, 'queue_time': 0.56766585, 'total_time': 0.118979606}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3760a355-9c6c-4a01-80fa-3376ba4055ea-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': '5jb87x0a0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 693, 'output_tokens': 28, 'total_tokens': 721})]

In [90]:
tool_result=multiply.invoke(result.tool_calls[0])

In [91]:
messages.append(tool_result)

In [92]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of multiplying 3 by 10 is 30.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 738, 'total_tokens': 752, 'completion_time': 0.03830883, 'prompt_time': 0.243332535, 'queue_time': 0.237398509, 'total_time': 0.281641365}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2eef0b05-71f9-42ed-a621-dadf970f6020-0', usage_metadata={'input_tokens': 738, 'output_tokens': 14, 'total_tokens': 752})

In [93]:
llm_with_tools.invoke(messages).content

'The result of multiplying 3 by 10 is 30.'